# m-LoRA: An Efficient "Factory" to Build Multiple LoRA Adapters
[![](https://github.com/mikecovlee/mLoRA/actions/workflows/python-test.yml/badge.svg)](https://github.com/mikecovlee/mLoRA/actions/workflows/python-test.yml)
[![](https://img.shields.io/github/stars/mikecovlee/mLoRA?logo=GitHub&style=flat)](https://github.com/mikecovlee/mLoRA/stargazers)
[![](https://img.shields.io/github/v/release/mikecovlee/mLoRA?logo=Github)](https://github.com/mikecovlee/mLoRA/releases/latest)
[![](https://img.shields.io/pypi/v/mlora?logo=pypi)](https://pypi.org/project/mlora/)
[![](https://img.shields.io/docker/v/mikecovlee/mlora?logo=Docker&label=docker)](https://hub.docker.com/r/mikecovlee/mlora/tags)
[![](https://img.shields.io/github/license/mikecovlee/mLoRA)](http://www.apache.org/licenses/LICENSE-2.0)

mLoRA (a.k.a Multi-LoRA Fine-Tune) is an open-source framework designed for efficient fine-tuning of multiple Large Language Models (LLMs) using LoRA and its variants. Key features of mLoRA include:

- Concurrent fine-tuning of multiple LoRA adapters.

- Shared base model among multiple LoRA adapters.

- Support for multiple LoRA variant algorithms and various base models.

- Exclusive Mo-LoRA (Mixture of LoRAs) optimization for MixLoRA and its variants.

## About this notebook

This is a simple jupiter notebook for showcasing the basic process of fine-tuning TinyLLaMA with dummy data

## Clone and install m-LoRA

In [ ]:
! pip install mlora

## Loading the base model

In [ ]:
import torch

import mlora

mlora.setup_logging("INFO")

base_model = "TinyLlama/TinyLlama_v1.1"

model = mlora.LLMModel.from_pretrained(
    base_model,
    device=mlora.get_backend().default_device_name(),
    load_dtype=torch.bfloat16,
)
tokenizer = mlora.Tokenizer(base_model)

## Training a dummy LoRA adapter

In [ ]:
lora_config = mlora.LoraConfig(
    adapter_name="lora_0",
    lora_r_=32,
    lora_alpha_=64,
    lora_dropout_=0.05,
    target_modules_={"q_proj": True, "k_proj": True, "v_proj": True, "o_proj": True},
)

model.init_adapter(lora_config)

train_config = mlora.TrainConfig(
    num_epochs=10,
    batch_size=16,
    micro_batch_size=8,
    learning_rate=1e-4,
    casual_train_data="TUDB-Labs/Dummy-mLoRA",
).init(lora_config)

mlora.train(model=model, tokenizer=tokenizer, configs=[train_config])

## Validate the effectiveness of LoRA adapter

In [ ]:
generate_config = mlora.GenerateConfig(
    adapter_name="lora_0",
    prompts=["Could you provide an introduction to m-LoRA?"],
    stop_token="\n",
)

output = mlora.generate(model=model, tokenizer=tokenizer, configs=[generate_config])

print(output["lora_0"][0])